# 计算输出特征图

**假设：**

> - 正方形的 方形的 输入 特征图 特征图 的大小为 的大小为 I * I 
> - 正方形的 卷积核的大小为 f * f
> - 步长 （stride）为 s
> - 填充 （padding）的行数或者列为 p

**问：**正方形的输出特征图的大小 （O * O）为多少？请列出相应的数学公式？

**答：**

$p = (I-f) \bmod s$

$O = \frac{I-f+2p}{s} + 1$

# 根据下图中描述的卷积神经网络参数搭建一个与之对应的卷积神经网络，实现 MNIST 手写数字识别

![](../pics/AIE20_W06_HW_CNN_01.png)

In [6]:
import tensorflow as tf
import os

In [7]:
lib_path = os.path.abspath('../libs/')
if lib_path not in os.sys.path:
    os.sys.path.append(lib_path)
import input_data

In [18]:
################################################################################
#
# load data
#
################################################################################
mnist = input_data.read_data_sets('mnist_data', one_hot=True)

Extracting mnist_data\train-images-idx3-ubyte.gz
Extracting mnist_data\train-labels-idx1-ubyte.gz
Extracting mnist_data\t10k-images-idx3-ubyte.gz
Extracting mnist_data\t10k-labels-idx1-ubyte.gz


In [31]:
################################################################################
#
# declare parameters & function
#
################################################################################

X_input = tf.placeholder(tf.float32,
                         shape=[None, 28, 28, 1])
Y_label = tf.placeholder(tf.float32,
                         shape=[None, 10])


def add_nn_layer(X,
                 in_size,
                 out_size,
                 activation_function=None):
    # weigths 使用随机正态分布初始化权重
    W = tf.Variable(tf.random_normal(shape=[in_size, out_size]))
    # bias 设置偏置单元的值为常量0.1
    b = tf.Variable(tf.constant(0.1, shape=[1, out_size]))
    # 计算：f = X product W + b
    f = tf.add(tf.multiply(X, W), b)

    return f if activation_function is None else activation_function(f)


def add_convolutional_layer(X,
                            in_channels,
                            out_channels,
                            filter_height,
                            filter_width,
                            stride,
                            activation_function=None):
    """添加卷积层

    Parameters:
    -----------
    X: 输入的数据 [batch, in_height, in_width, in_channels]
    in_channels: 输入特征图（feature map）的数量
    out_channels: 输出特征图（feature map）的数量
    filter_hight: 过滤器卷积核的高度
    filter_width：过滤器卷积核的宽度
    stride: 步长
    """
    # Filters 过滤器初始化
    F = tf.Variable(tf.truncated_normal(shape=[filter_height,
                                               filter_width,
                                               in_channels,
                                               out_channels]))
    # bais 偏置单元初始化
    b = tf.Variable(tf.constant(0.1, shape=[out_channels]))
    # X [batch, in_height, in_width, in_channels]
    # F [kernel_size kernel_size in_channels out_channels]
    conv = tf.nn.conv2d(X,
                        F,
                        strides=[1, stride, stride, 1],
                        padding='SAME')
    output = conv if activation_function is None else activation_function(conv)

    return tf.add(output, b)


def add_pooling_layer(X,
                      pool_height,
                      pool_width,
                      stride):
    """添加池化层

    Parameters:
    -----------
    X: 输入数据 [batch_size] + input_spatial_shape + [num_channels]
    pool_height: 池化核高度
    pool_width: 池化核宽度
    stride: 步长
    """
    return tf.nn.max_pool(X,
                          [1, pool_height, pool_width, 1],
                          [1, stride, stride, 1],
                          padding='SAME')

In [32]:
################################################################################
#
# initial tensorflow enviroment
#
################################################################################
tf.reset_default_graph()

In [33]:
################################################################################
#
# constructed graph
#
################################################################################
conv_1 = add_convolutional_layer(X_input,
                                 in_channels=1,
                                 out_channels=32,
                                 filter_height=5,
                                 filter_width=5,
                                 stride=2,
                                 activation_function=tf.nn.relu)

ValueError: Tensor("Variable:0", shape=(5, 5, 1, 32), dtype=float32_ref) must be from the same graph as Tensor("Placeholder:0", shape=(?, 28, 28, 1), dtype=float32).

In [ ]:
################################################################################
#
# executed graph
#
################################################################################